## PROJECT GOAL:
Outbrain is a web advertising platform that displays advertisement boxes of links to pages within websites. It displays links to the sites' pages in addition to sponsored content, generating revenue from the latter.
In our project we are predicting whether the particular ad_id will be clicked or not.

This python files predict ad using random forest classifier algorithm and measure Accuracy and AUC.

## Creating a Classification Model using Logistic Regression Algorithm

In this file, we will implement a classification model using *Logistic Regression that uses features of a Event,Document and Train dataframe to **predict the whether the ad is clicked or not** 

You should follow the steps below to build, train and test the model from the source data:

1. Build a schema of a source data for its Data Frame
2. Load the Source Data to the schema
3. Prepare the data with the features (input columns, output column as label)
4. Split the data using data.randomSplit(): Training and Testing
5. Transform the columns to a vector using VectorAssembler
6. set features and label from the vector
7. Build a ***Logistic Regression Algorithm*** Model with the label and features
8. Train the model
9. Prepare the testing Data Frame with features and label from the vector; Rename label to trueLabel
10. Predict and test the testing Data Frame using the model trained at the step 8
11. Compare the predicted result and trueLabel


### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [3]:
# Import Spark SQL and Spark ML libraries
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.mllib.tree import RandomForest
from pyspark.ml import Pipeline

from pyspark.ml.classification import DecisionTreeClassifier

from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier



#####The below command will start spark session when we run our file in oracle BDE. In Databricks keep this cell as False by default. But when you run file in Oracle BDE make it True.

In [5]:
IS_SPARK_SUBMIT_CLI = False
if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

### Load Source Data
The data for this project is provided as a CSV file containing details of advertisement. The data includes specific characteristics (or *features*) for each ad, as well as a *label* column indicating whether the ad was clicked or not.

You will load this data into a DataFrame and display it.

#####Reading all necessary csv file

In [8]:
if IS_SPARK_SUBMIT_CLI:
    event = spark.read.csv('eventd.csv', inferSchema=True, header=True)
else:
    event = spark.sql("SELECT * FROM eventd_csv")

In [9]:
if IS_SPARK_SUBMIT_CLI:
    train = spark.read.csv('traind.csv', inferSchema=True, header=True)
else:
    train = spark.sql("SELECT * FROM traind_csv")

In [10]:
if IS_SPARK_SUBMIT_CLI:
    promoted_content = spark.read.csv('promoted_content.csv', inferSchema=True, header=True)
else:
    promoted_content = spark.sql("SELECT * FROM promoted_content_csv")

Below command will join event and train file using disply_id as primary key. It will also drop display_id from event table and ad_id from train table as it create ambiguity error. Then it will assign result to new dataframe data1.

In [12]:
data1=event.join(train,event.display_id==train.display_id).drop(event.display_id).drop(train.ad_id)

Following cell will join two dataframe data1 (which created from joining of event and train table) with Promoted content dataframe using document_id as primary key. Then it will assign result to new dataframe data2.

In [14]:
data2=data1.join(promoted_content,data1.document_id==promoted_content.document_id).drop(data1.document_id)

Following table select disply_id,document_id,platform,ad_id,camaign_id,advertiser_id which are features which help us to train our dataset and clicked column which is label column. Selected column will assign to new dataframe called data.

In [16]:
data = data2.select("display_id", "document_id", "platform", "ad_id", "campaign_id","advertiser_id",col("clicked").alias("label"))

### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [18]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

### Prepare the Training Data create Pipeline
To train the classification model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **VectorAssembler** class to transform the feature columns into a vector,Normlize data using Minmax.

In [20]:
from pyspark.ml.classification import RandomForestClassifier
assembler = VectorAssembler(inputCols = ["display_id", "document_id", "platform", "ad_id","campaign_id","advertiser_id"],outputCol="normfeatures")
#assembler = VectorAssembler(inputCols = ["clicked"],outputCol="label")
minMax = MinMaxScaler(inputCol = assembler.getOutputCol(), outputCol="nfeatures")
featVect = VectorAssembler(inputCols=["nfeatures"], outputCol="features")
#rf = RandomForestClassifier(labelCol="label", featuresCol="features",impurity="gini",featureSubsetStrategy="auto",numTrees=10,maxDepth=32,maxBins=128,seed=1234)
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
pipeline = Pipeline(stages=[assembler,minMax,featVect,lr])

### Tune Parameters
You can tune parameters to find the best model for your data. To do this you can use the  **CrossValidator** class to evaluate each combination of parameters defined in a **ParameterGrid** against multiple *folds* of the data split into training and validation datasets, in order to find the best performing parameters. Note that this can take a long time to run because every parameter combination is tried multiple times.

In [22]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.01]).addGrid(lr.maxIter, [10, 5]).build()
# TODO: K = 2, you may test it with 5, 10
# K=2, 5, 10: Root Mean Square Error (RMSE): 13.2
cv = CrossValidator(estimator=pipeline, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid, numFolds=10)

model = cv.fit(train)


MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

### Test the Recommender
The model produced by the pipeline is a transformed that will apply to all stages in the pipeline to a specified DataFrame and apply the trained model to generate predictions. In this case, you will transform the test DataFrame using the pipeline to generate label prediction

In [24]:
# piplineModel with train data set applies test data set and generate predictions
prediction = model.transform(test)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show(100, truncate=False)

+----------------------------------------------------------------------------------------------------------+----------+---------+
features |prediction|trueLabel|
+----------------------------------------------------------------------------------------------------------+----------+---------+
[0.23366817877427618,0.5000774396069431,0.0,0.28593653424734505,0.5789429127747834,0.38717002554640934] |0.0 |1 |
[0.23366817877427618,0.5000774396069431,0.0,0.3179766955104793,0.5789429127747834,0.38717002554640934] |0.0 |1 |
[0.23366817877427618,0.5000774396069431,0.0,0.31797845142439735,0.5789429127747834,0.38717002554640934] |0.0 |1 |
[0.23366817877427618,0.5000774396069431,0.0,0.3179854750800697,0.5789429127747834,0.38717002554640934] |0.0 |1 |
[0.7016187812154383,0.756300954993102,1.0,0.7864966707872113,0.34760278804639216,0.08174850979279023] |0.0 |0 |
[0.10106117287015641,0.27458689636005235,0.5,0.07753237905264933,0.14343764991393176,0.01107011070110701] |0.0 |0 |
[0.10106117287015641,0.27458689636005235,0.5,0.0775411586222397,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.10106117287015641,0.27458689636005235,0.5,0.07754291453615778,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.18811473807436094,0.27458689636005235,0.5,0.0775341349665674,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.18811473807436094,0.27458689636005235,0.5,0.07753940270832163,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.18811473807436094,0.27458689636005235,0.5,0.07844369837613081,0.001890679233569433,0.01107011070110701]|0.0 |0 |
[0.41789857287885424,0.12195969062787604,1.0,0.007994676069000393,0.03036374410926433,0.01845018450184502]|0.0 |0 |
[0.524338853152041,0.27458689636005235,0.0,0.07753237905264933,0.14343764991393176,0.01107011070110701] |0.0 |1 |
[0.524338853152041,0.27458689636005235,0.0,0.07753940270832163,0.05909078082230437,0.01107011070110701] |0.0 |1 |
[0.6057386124259493,0.27458689636005235,1.0,0.07753237905264933,0.14343764991393176,0.01107011070110701] |0.0 |1 |
[0.6057386124259493,0.27458689636005235,1.0,0.0775341349665674,0.05909078082230437,0.01107011070110701] |0.0 |1 |
[0.6057386124259493,0.27458689636005235,1.0,0.07753940270832163,0.05909078082230437,0.01107011070110701] |0.0 |1 |
[0.6057386124259493,0.27458689636005235,1.0,0.0775411586222397,0.05909078082230437,0.01107011070110701] |0.0 |1 |
[0.6679213408183441,0.8870200845790737,0.5,0.8874406501095691,0.02342184722183029,0.04598353675844451] |0.0 |0 |
[0.6679213408183441,0.8870200845790737,0.5,0.8877356436478058,0.0027372520247199257,0.04598353675844451] |0.0 |0 |
[0.9036495805567077,0.27458689636005235,1.0,0.0775341349665674,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9036495805567077,0.27458689636005235,1.0,0.07753940270832163,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9036495805567077,0.27458689636005235,1.0,0.0775411586222397,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9036495805567077,0.27458689636005235,1.0,0.07754291453615778,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9450505050744769,0.27458689636005235,1.0,0.0775341349665674,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9450505050744769,0.27458689636005235,1.0,0.07753589088048547,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9450505050744769,0.27458689636005235,1.0,0.07753940270832163,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9450505050744769,0.27458689636005235,1.0,0.07754291453615778,0.05909078082230437,0.01107011070110701] |0.0 |0 |
[0.9450505050744769,0.27458689636005235,1.0,0.07844369837613081,0.001890679233569433,0.01107011070110701] |0.0 |0 |
[0.07619282792403678,0.6488977643382204,1.0,0.6305293729280216,0.006885458701357338,2.838489923360772E-4] |0.0 |0 |
[0.5435174974358998,0.4825349542355173,0.0,0.2679770466932629,0.13502836018850353,0.4175418677263696] |0.0 |0 |
[0.9411250453208072,0.4825349542355173,0.0,0.2679770466932629,0.13502836018850353,0.4175418677263696] |0.0 |0 |
[0.018739304630639945,0.0419000138425533,0.0,

Following cell measure accuracy of Algorithm using MultiClassificationEvaluater and evaluate using predicted data.

In [26]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluation = MulticlassClassificationEvaluator(
    labelCol="trueLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluation.evaluate(prediction)
print("Accuracy of Logistic Regression is: ",accuracy)

Accuracy of Logistic Regression is: 0.8530731429385371

Following Code will give total Error in project

In [28]:
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.146927

#### Logistic Regression Evaluator
Calculate AUC using Logistic Regression Evaluator.

In [30]:
rf_evaluator =  MulticlassClassificationEvaluator(labelCol="trueLabel", predictionCol="prediction")
rf_auc = rf_evaluator.evaluate(prediction)
print("AUC for Logistic Regression is= ", rf_auc)

AUC for Logistic Regression is= 0.7854344983371995

#### Logistic Regression Confusion matrix
Calculate confusion matrix and measure precision,Recall

In [32]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp ),
 ("Recall", tp / (tp + fn))],["metric", "value"])
metrics.show()


+---------+-------+
 metric| value|
+---------+-------+
 TP| 0.0|
 FP| 0.0|
 TN|11978.0|
 FN| 2063.0|
Precision| 0.0|
 Recall| 0.0|
+---------+-------+